In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
import json
import matplotlib.pyplot as plt
import numpy as np

import recova.learning_dataset
import recova.trajectory_evaluation as traj
from recova.registration_result_database import RegistrationPairDatabase
from recova.learning.learning import model_from_file
from recova.util import kullback_leibler, wishart_kl_divergence

np.set_printoptions(precision=3, linewidth=100)

In [3]:
locations = {
    'apartment': (
        '/home/dlandry/dataset/learning_sets/2018-09-11-inside.json', 
        '/home/dlandry/dataset/learning_runs/2018-09-11-apartment.json',
        '/home/dlandry/dataset/learning_runs/2018-09-11-apartment.model',
        '/home/dlandry/dataset/censi_estimates/2018-09-11-apartment.json'
    ), 'hauptgebaude': (
        '/home/dlandry/dataset/learning_sets/2018-09-11-inside.json', 
        '/home/dlandry/dataset/learning_runs/2018-09-11-hauptgebaude.json', 
        '/home/dlandry/dataset/learning_runs/2018-09-11-hauptgebaude.model', 
        '/home/dlandry/dataset/censi_estimates/2018-09-11-hauptgebaude.json'
    ), 'stairs': (
        '/home/dlandry/dataset/learning_sets/2018-09-11-inside.json',
        '/home/dlandry/dataset/learning_runs/2018-09-11-stairs.json', 
        '/home/dlandry/dataset/learning_runs/2018-09-11-stairs.model', 
        '/home/dlandry/dataset/censi_estimates/2018-09-11-stairs.json'
    ), 'gazebo_summer': (
        '/home/dlandry/dataset/learning_sets/2018-09-12-gazebo.json',
        '/home/dlandry/dataset/learning_runs/2018-09-12-gazebo-summer.json',
        '/home/dlandry/dataset/learning_runs/2018-09-12-gazebo-summer.model',
        '/home/dlandry/dataset/censi_estimates/2018-09-12-gazebo-summer.json'
    ), 'gazebo_winter': (
        '/home/dlandry/dataset/learning_sets/2018-09-12-gazebo.json',
        '/home/dlandry/dataset/learning_runs/2018-09-12-gazebo-winter.json',
        '/home/dlandry/dataset/learning_runs/2018-09-12-gazebo-winter.model',
        '/home/dlandry/dataset/censi_estimates/2018-09-12-gazebo-winter.json'
    ), 'wood_autumn': (
        '/home/dlandry/dataset/learning_sets/2018-09-12-wood.json',
        '/home/dlandry/dataset/learning_runs/2018-09-12-wood-autumn.json',
        '/home/dlandry/dataset/learning_runs/2018-09-12-wood-autumn.model',
        '/home/dlandry/dataset/censi_estimates/2018-09-12-wood-autumn.json'
    ), 'wood_summer': (
        '/home/dlandry/dataset/learning_sets/2018-09-12-wood.json',
        '/home/dlandry/dataset/learning_runs/2018-09-12-wood-summer.json', 
        '/home/dlandry/dataset/learning_runs/2018-09-12-wood-summer.model',
        '/home/dlandry/dataset/censi_estimates/2018-09-12-wood-summer.json'
    ),
}


In [4]:
def frobenius_norm(m):
    return np.sqrt(np.trace((m).T @ (m)))

In [5]:
def dataset_to_loss(learning_set_file, learning_run_file, model_file, censi_estimates_file):
    with open(learning_set_file) as f:
        dataset = json.load(f)
    
    with open(learning_run_file) as f:
        learning_run = json.load(f)

    with open(censi_estimates_file) as f:
        censi_estimates = json.load(f)
    
    model = model_from_file(model_file, 'cello')
    
    xs = np.array(dataset['data']['xs'])
    ys = np.array(dataset['data']['ys'])


    train_set = learning_run['train_set']
    validation_set = learning_run['validation_set']
    
    validation_xs = xs[validation_set]
    validation_ys = ys[validation_set]

    train_ys = ys[train_set]
    avg_train_ys = np.mean(train_ys, axis=0)
    
    predictions = model.predict(validation_xs)
    censi_predictions = np.array(censi_estimates['data']['censi_estimates'])
    
    reference_norms = np.zeros(len(predictions))
    prediction_norms = np.zeros(len(predictions))
    norm_loss = np.zeros(len(predictions))
    gaussian_kl = np.zeros(len(predictions))
    percent_loss = np.zeros(len(predictions))
    censi_loss = np.zeros(len(predictions))
    censi_kl = np.zeros(len(predictions))
    baseline_loss = np.zeros(len(predictions))
    baseline_kl = np.zeros(len(predictions))

    for i in range(len(predictions)):
        reference_norms[i] = np.linalg.norm(ys[validation_set[i]])
        prediction_norms[i] = np.linalg.norm(predictions[i])
        norm_loss[i] = frobenius_norm(ys[validation_set[i]] - predictions[i])
        percent_loss = abs(norm_loss[i] / np.linalg.norm(ys[validation_set[i]]))
        gaussian_kl[i] = kullback_leibler(ys[validation_set[i]], predictions[i])

        censi_loss[i] = frobenius_norm(ys[validation_set[i]] - censi_predictions[i])
        censi_kl[i] = kullback_leibler(ys[validation_set[i]], censi_predictions[i])

        baseline_loss[i] = frobenius_norm(ys[validation_set[i]] - avg_train_ys)
        baseline_kl[i] = kullback_leibler(ys[validation_set[i]], avg_train_ys)
            
    print('{}, {}, {:.3f}, {:.3f}, {:.2E}'.format(
        learning_run['metadata']['cross_validation'],
        len(predictions),
        baseline_kl.mean(),
        gaussian_kl.mean(),
        censi_kl.mean()
    ))
        
    print()
    print()
    print('Dataset: {}'.format(learning_run['metadata']['cross_validation']))
    print('N Pairs: {}'.format(len(predictions)))

    print('Avg. Percent loss: {:.2E}'.format(percent_loss.mean()))
    print()
    print('Avg. Baseline loss: {:.4E}'.format(baseline_loss.mean()))
    print('Avg. Baseline KL: {:.2f}'.format(baseline_kl.mean()))
    print()
    print('Avg. loss: {:.2E}'.format(norm_loss.mean()))
    print('Avg. KL loss: {:.2f}'.format(gaussian_kl.mean()))
    print()
    print('Avg. Censi Loss: {:.2E}'.format(censi_loss.mean()))
    print('Avg. Censi KL loss: {:.2E}'.format(censi_kl.mean()))
    print()

In [11]:
def dispersion_of_model(model_file):
    model = model_from_file(model_file, 'cello')
    

# Dispersion of model

In [28]:
model = model_from_file(locations['apartment'][2], 'cello')



In [30]:
model.theta.std().item()

0.012994935736060143

In [31]:
model.theta_to_metric_matrix(model.theta).std().item()

0.03357156366109848

In [40]:
for loc in sorted(locations):
    model = model_from_file(locations[loc][2], 'cello')
    
    print('{:<13}: {:03f}'.format(loc, model.theta_to_metric_matrix(model.theta).std().item()))

apartment    : 0.033572
gazebo_summer: 0.002597
gazebo_winter: 0.003647
hauptgebaude : 0.036582
stairs       : 0.036103
wood_autumn  : 0.018713
wood_summer  : 0.018524


In [21]:
dataset_to_loss(*locations['wood_summer'])

wood_summer, 966, 13.607, 11.287, 3.52E+07


Dataset: wood_summer
N Pairs: 966
Avg. Percent loss: 8.58E-01

Avg. Baseline loss: 2.6320E-02
Avg. Baseline KL: 13.61

Avg. loss: 2.00E-02
Avg. KL loss: 11.29

Avg. Censi Loss: 2.07E-02
Avg. Censi KL loss: 3.52E+07

